In [1]:
!mkdir -p binfo1-project/data
# >> 여기까지 리눅스(Ubuntu) shell에서 실행 (느낌표 떼고)
# 다운로드 받은 데이터셋을 로컬 binfo1-project 폴더에 직접 업로드

%ls -al binfo1-project
%cd binfo1-project/data

!gunzip hg19_v19_annotation.gtf.gz
!gunzip goa_human.gaf.gz
!Homo_sapiens.GRCh38.109.gtf.gz

!conda install -c bioconda subread

!featureCounts -a hg19_v19_annotation.gtf -o read-counts.txt *.bam
# >> 여기까지 리눅스(Ubuntu) shell에서 실행

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats

cnts = pd.read_csv('read-counts.txt', sep='\t', comment='#', index_col=0)
cnts.head()

In [ ]:
cnts['clip_enrichment'] = cnts['eCLIP_H_RBFOX2XPT.bam'] / cnts['eCLIP_control_H_RBFOX2XPT.bam']
cnts['transcr_change'] = cnts['shRNA_KD_RBFOX2.bam'] / cnts['shRNA_control_RBFOX2.bam']
cnts.head(n=20)

In [ ]:
cnts_excluded = cnts[(cnts['eCLIP_H_RBFOX2XPT.bam'] >= 30) & (cnts['eCLIP_control_H_RBFOX2XPT.bam'] >= 30) & (cnts['shRNA_KD_RBFOX2.bam'] >= 30) & (cnts['shRNA_control_RBFOX2.bam'] >= 30)].copy()
cnts_excluded.head(n=20)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.scatter(np.log2(cnts_excluded['clip_enrichment']), 
           np.log2(cnts_excluded['transcr_change']), 
           s=1, c='#000000', alpha=0.15)

ax.set_title('Correlation between CLIP enrichment and \nRNA level upon RBFOX2 knockdown')
ax.set_xlabel('RBFOX2 CLIP Enrichment (log$_2$)')
ax.set_ylabel('mRNA abundance change upon RBFOX2 knockdown (log$_2$)')

r, p = stats.pearsonr(np.log2(cnts_excluded['clip_enrichment']), np.log2(cnts_excluded['transcr_change']))
ax.annotate('r = {:.2f}'.format(r), xy=(0.8, 0.05), xycoords='axes fraction')

#plt.show()
plt.savefig('CLIP_enrichment_and_Transcription.png')

# >> 여기까지 binfo1.py file로 만들어서 저장 후 리눅스(Ubuntu) shell에서 실행
# 결과파일 - CLIP_enrichment_and_Transcription.png

In [ ]:
!sed 's/\"//g' Homo_sapiens_GRCh37.gtf | sed "s/^.*gene_id \([^;]*\);.*gene_name \([^;]*\).*/\1\t\2/g" > gene_name.txt
# 여기서 정규표현식이 의도대로 작동하지 않아 Sublime text에서 수정함 (UniProt gene ID를 Ensembl gene ID로 바꿔주기 위한 목록)

!echo -e "gene_id\tGene names\ttype" > goa_locate.txt
!awk '$5 == "GO:0005634" { print $2"\t"$3"\tnucleus" }' goa_human.gaf >> goa_locate.txt # goa_human = UniProt-GOA DB (hg19)
!awk '$5 == "GO:0016021" { print $2"\t"$3"\tintegral membrane" }' goa_human.gaf >> goa_locate.txt
!awk '$5 == "GO:0005737" { print $2"\t"$3"\tcytoplasm" }' goa_human.gaf >> goa_locate.txt 
!awk '{ print $2"\t"$3"\tintegral membrane" }' QuickGO-annotations.gaf >> goa_locate.txt 
# QuickGO annotations = QuickGO site에서 다운로드 받은 추가 database. Integral membrane(GO:0016021)에 속한 gene 중 no evidence data, automatic assertion 제외함

!awk '$5 == "GO:0031966" { print $2 }' goa_human.gaf > erase.txt

# >> 여기까지 리눅스(Ubuntu) shell에서 실행

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats

df_local = pd.read_csv('goa_locate.txt', sep='\t')

with open('erase.txt') as f:
    for line in f.readlines():
      if line in df_local.index.to_list():
        df_local.drop(index=line)

df_local.to_csv('goa_locate.txt', sep = '\t')

gene_dict={}
with open('gene_name.txt') as f:
    lines = f.readlines()
    for name in lines:
      gene_dict[name.split('\t')[1].strip()] = name.split('\t')[0].strip()

local_dict={}
with open('goa_locate.txt') as f:
  for line in f.readlines():
    gene_name = line.split('\t')[1]
    gene_locus = line.split('\t')[2]
    if gene_name in list(gene_dict.keys()):
      local_dict[gene_dict[gene_name]]=gene_locus


In [ ]:
local_list=[]
for i, items in enumerate(cnts_excluded.iterrows()):
  gene = items[0].split('.')[0]
  if gene in local_dict.keys():
    local = local_dict[gene]
    local_list.append(local)
  else:
    local_list.append('not_in_list')
print(local_list)

cnts_excluded['local']=local_list
print(cnts_excluded)

cnts_localized = cnts_excluded[cnts_excluded.local != 'not_in_list'].copy()
print(cnts_localized)

In [ ]:
color_dict = {'integral membrane':'#e32b51', 'cytoplasm':'#50b846', 'nucleus':'#546fb6'}

fig, ax = plt.subplots(1, 1, figsize=(5, 5))

for local in ['integral membrane','cytoplasm','nucleus']:
  cnts_plot=cnts_localized[cnts_localized['local']==local]
  ax.scatter(np.log2(cnts_plot['clip_enrichment']), 
            np.log2(cnts_plot['transcr_change']), 
            s=1, alpha=0.15,
            c=color_dict[local], label=local)

ax.set_title('Correlation between CLIP enrichment and \nRNA level upon RBFOX2 knockdown')
ax.set_xlabel('RBFOX2 CLIP Enrichment (log$_2$)')
ax.set_ylabel('mRNA abundance change upon RBFOX2 knockdown (log$_2$)')

r, p = stats.pearsonr(np.log2(cnts_localized['clip_enrichment']), np.log2(cnts_localized['transcr_change']))
ax.annotate('r = {:.2f}'.format(r), xy=(0.8, 0.05), xycoords='axes fraction')
ax.legend(markerscale=3, loc='lower left', fontsize=6)

#plt.show()
plt.savefig('CLIP_enrichment_localization.png')

# >> 여기까지 binfo2.py file로 만들어서 저장 후 리눅스(Ubuntu) shell에서 실행
# 결과파일 - CLIP_enrichment_localization.png 
# database 추가 전 결과 파일 (integral membrane localization 없음) = CLIP_enrichment_localization_before.png

In [ ]:
!conda install -c bioconda -y bedtools bioawk samtools
!grep -iP "PCBP1\"" Homo_sapiens.GRCh37.87.gtf
# >> 여기까지 리눅스(Ubuntu) shell에서 실행
# 결과 중, exon & transcript의 범위 = chr2:70314585-70316332

In [ ]:
!samtools index eCLIP_H_RBFOX2XPT.bam
!samtools view -b -o eCLIP-PCBP1.bam eCLIP_H_RBFOX2XPT.bam chr2:70314585-70316332
!samtools view eCLIP-PCBP1.bam | wc -l
!samtools mpileup eCLIP-PCBP1.bam > eCLIP-PCBP1.pileup
!wc -l eCLIP-PCBP1.pileup

!awk '$2 >= 70314585 && $2 <= 70316332 { print $0; }' eCLIP-PCBP1.pileup > eCLIP-PCBP1.pileup
!cat eCLIP-PCBP1.pileup
# >> 여기까지 리눅스(Ubuntu) shell에서 실행

In [ ]:
import pandas as pd
import re

pileup = pd.read_csv('eCLIP-PCBP1.pileup', sep='\t', names=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals'])
pileup.tail()

toremove = re.compile('[1-9nN<>\$\*#-]|\^~[A-Za-z]')
pileup['matches'] = pileup['basereads'].apply(lambda x: toremove.sub('', x))

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth', 1000)
pileupdf = pileup[['chrom', 'pos', 'count', 'matches']].copy()

print(pileupdf)
# >> 여기까지 binfo3.py file로 만들어서 저장 후 리눅스(Ubuntu) shell에서 실행
# pileupdf 형태 확인 - 특수문자 제거는 잘 되었으나 대소문자 섞여있고 read number가 낮은 편

In [ ]:
import pandas as pd
import numpy as np

base_list=['A','G','C','T']
for i in range(len(pileupdf)):
  matches = pileupdf.iloc[i]['matches']
  if not matches or pileupdf.iloc[i].isnull().sum():
    print(i, 'no match')
    pass
  else:
    Shannon=0
    p_list=[]
    for base in base_list:
      p_base = matches.upper().count(base)/len(matches)
      p_list.append(p_base)
      if p_base != 0:
        Shannon+=-p_base*np.log2(p_base)
    print(i, p_list, Shannon)
# >> 여기까지 binfo4.py file로 만들어서 저장 후 리눅스(Ubuntu) shell에서 실행
# Shannon entropy 값 확인 - OK

In [ ]:
import pandas as pd
import numpy as np

base_list=['A','G','C','T']
for i in range(len(pileupdf)):
  matches = pileupdf.iloc[i]['matches']
  if not matches or pileupdf.iloc[i].isnull().sum():
    pass
  else:
    Shannon=0
    p_list=[]
    for base in base_list:
      p_base = matches.upper().count(base)/len(matches)
      p_list.append(p_base)
      if p_base != 0:
        Shannon+=-p_base*np.log2(p_base)
    if Shannon > 0.7:
      print(i, p_list, Shannon)
# >> 여기까지 binfo5.py file로 만들어서 저장 후 리눅스(Ubuntu) shell에서 실행
# Shannon entropy > 0.7인 position 확인 - 24개 locus

In [ ]:
import pandas as pd
import numpy as np

base_list=['A','G','C','T']
for i in range(len(pileupdf)):
  matches = pileupdf.iloc[i]['matches']
  if not matches or pileupdf.iloc[i].isnull().sum():
    pass
  else:
    Shannon=0
    p_list=[]
    for base in base_list:
      p_base = matches.upper().count(base)/len(matches)
      p_list.append(p_base)
      if p_base != 0:
        Shannon+=-p_base*np.log2(p_base)
    if Shannon > 0.7 and len(matches) > 3:
      print(i, p_list, Shannon)
# >> 여기까지 binfo6.py file로 만들어서 저장 후 리눅스(Ubuntu) shell에서 실행
# Shannon entropy > 0.7, read number > 3인 position 확인 - 5개 locus
'''
68 [0.0, 0.0, 0.75, 0.25] 0.8112781244591328
88 [0.2, 0.8, 0.0, 0.0] 0.7219280948873623
110 [0.25, 0.0, 0.75, 0.0] 0.8112781244591328
715 [0.2, 0.0, 0.0, 0.8] 0.7219280948873623
738 [0.4, 0.0, 0.0, 0.6] 0.9709505944546686
'''

In [ ]:
import pandas as pd
import numpy as np

base_list=['A','G','C','T']
for i in range(len(pileupdf)):
  matches = pileupdf.iloc[i]['matches']
  if not matches or pileupdf.iloc[i].isnull().sum():
    print(pileupdf.iloc[i]['chrom'], pileupdf.iloc[i]['pos'], pileupdf.iloc[i]['pos']+1, 0)
    pass
  else:
    Shannon=0
    p_list=[]
    for base in base_list:
      p_base = matches.upper().count(base)/len(matches)
      p_list.append(p_base)
      if p_base != 0:
        Shannon+=-p_base*np.log2(p_base)
    print(pileupdf.iloc[i]['chrom'], pileupdf.iloc[i]['pos'], pileupdf.iloc[i]['pos']+1, Shannon)

# >> 여기까지 binfo7.py file로 만들어서 저장 후 리눅스(Ubuntu) shell에서 실행
# 결과 파일 복사해서 UCSC Genome Browser - hg19 reference에 입력해 비교